In [1]:
import gurobipy as gp
from gurobipy import GRB

import json

# Read capital names and coordinates from json file
try:
  capitals_json = json.load(open('capitals.json'))
except:
  import urllib.request
  url = 'https://raw.githubusercontent.com/eren-berik/TSP_Tasarim2/main/capitals.json'
  data = urllib.request.urlopen(url).read()
  capitals_json = json.loads(data)

capitals = []
coordinates = {}
for state in capitals_json:
    capital = capitals_json[state]['capital']
    capitals.append(capital)
    coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

In [2]:
import math
from itertools import combinations

# Compute pairwise distance matrix

import math

# Distance between cities, using Haversine formula
def distance(city1, city2):
    # Convert coordinates to radians
    lat1 = math.radians(coordinates[city1][0])
    lon1 = math.radians(coordinates[city1][1])
    lat2 = math.radians(coordinates[city2][0])
    lon2 = math.radians(coordinates[city2][1])
    
    # Calculate the difference between the coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Apply the Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    # Return the distance in kilometers
    return 6371 * c

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

In [3]:
print(dist)

{('Montgomery', 'Juneau'): 4589.225752409685, ('Montgomery', 'Phoenix'): 2404.970627015125, ('Montgomery', 'Little Rock'): 619.7330222422992, ('Montgomery', 'Sacramento'): 3240.0541185343413, ('Montgomery', 'Denver'): 1865.9828946727846, ('Montgomery', 'Hartford'): 1593.429590673051, ('Montgomery', 'Dover'): 1228.5232821667344, ('Montgomery', 'Tallahassee'): 285.20470166877124, ('Montgomery', 'Atlanta'): 234.88316269028277, ('Montgomery', 'Montgomery_2'): 0.00014555483126556793, ('Juneau', 'Phoenix'): 3226.716523289665, ('Juneau', 'Little Rock'): 4043.745923680031, ('Juneau', 'Sacramento'): 2384.982789547333, ('Juneau', 'Denver'): 2934.0418789789173, ('Juneau', 'Hartford'): 4582.530307229883, ('Juneau', 'Dover'): 4625.749619886668, ('Juneau', 'Tallahassee'): 4872.9333065164055, ('Juneau', 'Atlanta'): 4571.311852004465, ('Juneau', 'Montgomery_2'): 4589.225607174491, ('Phoenix', 'Little Rock'): 1820.698818478143, ('Phoenix', 'Sacramento'): 1017.3464945273328, ('Phoenix', 'Denver'): 942.6

In [4]:
vehicles = ['vehicle_1','vehicle_2']

# Create an empty model
m = gp.Model()

# Define the decision variables
# Binary variable indicating whether a city is visited by a vehicle
visited = m.addVars(capitals, vtype=GRB.BINARY, name='visited')

# Binary variable indicating whether a vehicle is at a city at a particular time
at_city = m.addVars(capitals, vehicles, vtype=GRB.BINARY, name='at_city')

# Continuous variable indicating the time at which a vehicle arrives at a city
arrival_time = m.addVars(capitals, vehicles, name='arrival_time')

# Define the objective function
# Minimize the total distance traveled by the vehicles
m.setObjective(gp.quicksum(dist[i, j] * at_city[i, v] * at_city[j, v]
                           for v in vehicles
                           for i, j in combinations(capitals, 2)
                           if i != j),
               sense=GRB.MINIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-02


In [5]:
# Define the constraints

# Each city must be visited by exactly one vehicle
for c in capitals[1:10]:
    m.addConstr(gp.quicksum(at_city[c, v] for v in vehicles) == 1)

# The starting city must be visited by all vehicles
for v in vehicles:
    m.addConstr(at_city[capitals[0], v] == 1)

# The ending city must be visited by all vehicles
for v in vehicles:
    m.addConstr(at_city[capitals[-1], v] == 1)

# The arrival time at each city must be greater than or equal to the previous arrival time
for c in capitals[1:]:
    for v in vehicles:
        m.addConstr(arrival_time[c, v] >= arrival_time[capitals[0], v] +
                    gp.quicksum(dist[capitals[0], i] * at_city[i, v] for i in capitals[1:]))

# The arrival time at the starting city must be 0
for v in vehicles:
    m.addConstr(arrival_time[capitals[0], v] == 0)

# The arrival time at the ending city must be greater than or equal to starting city
for v in vehicles:
    m.addConstr(arrival_time[capitals[-1], v] >= arrival_time[capitals[0], v])


# Set the LazyConstraints parameter to 1
m.setParam(GRB.Param.LazyConstraints, 1)

# Add subtour elimination constraints dynamically
def subtour_elimination(model, where):
    if where == GRB.callback.MIPSOL:
        # Get the solution
        x = model.cbGetSolution(at_city)
        visited = [c for c in capitals if x[c, v] > 0.5]
        unvisited = [c for c in capitals if c not in visited]

        # Check if all cities have been visited
        if len(visited) == len(capitals):
            return
        else:
            # Add a constraint to exclude the current solution
            model.cbLazy(gp.quicksum(at_city[c, v] for c in unvisited) <= len(unvisited) - 1)


Set parameter LazyConstraints to value 1


In [6]:
# Optimize the model
m.optimize(subtour_elimination)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 55 columns and 268 nonzeros
Model fingerprint: 0x62d7d38f
Model has 110 quadratic objective terms
Variable types: 22 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [1e-04, 5e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e-04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 17 rows and 15 columns
Presolve time: 0.00s
Presolved: 56 rows, 76 columns, 308 nonzeros
Variable types: 20 continuous, 56 integer (56 binary)
Found heuristic solution: objective 63813.901497

Root relaxation: objective 3.212401e+04, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [7]:
# Print the optimal solution
if m.status == GRB.OPTIMAL:
    print ('\nSolverResult:', m.objVal)

    for c in capitals:
        if visited[c].x > 0.001:
            print("visited(%s)" % (c), visited[c].x)
        print("\n")
            
        for v in vehicles:
            print(f'Vehicle {v}:')
            if at_city[c, v].x > 0.5:
                print(f'  {c} ({arrival_time[c, v].x:.2f})')
            
else:
    print('Optimal solution not found')


SolverResult: 55116.50235887667


Vehicle vehicle_1:
  Montgomery (0.00)
Vehicle vehicle_2:
  Montgomery (0.00)


Vehicle vehicle_1:
  Juneau (12100.23)
Vehicle vehicle_2:


Vehicle vehicle_1:
  Phoenix (12100.23)
Vehicle vehicle_2:


Vehicle vehicle_1:
Vehicle vehicle_2:
  Little Rock (6366.74)


Vehicle vehicle_1:
  Sacramento (12100.23)
Vehicle vehicle_2:


Vehicle vehicle_1:
  Denver (12100.23)
Vehicle vehicle_2:


Vehicle vehicle_1:
Vehicle vehicle_2:
  Hartford (6366.74)


Vehicle vehicle_1:
Vehicle vehicle_2:
  Dover (6366.74)


Vehicle vehicle_1:
Vehicle vehicle_2:
  Tallahassee (6366.74)


Vehicle vehicle_1:
Vehicle vehicle_2:
  Atlanta (6366.74)


Vehicle vehicle_1:
  Montgomery_2 (12100.23)
Vehicle vehicle_2:
  Montgomery_2 (6366.74)


In [8]:
# Print the total distance traveled by each vehicle
for v in vehicles:
    route = [capitals[0]]
    for c in capitals[1:]:
        if at_city[c, v].x > 0.5:
            route.append(c)
    print(f'Total distance for {v}: {sum(distance(route[i], route[i+1]) for i in range(len(route)-1))} km')
    print(f'Route for {v}: {route}')

Total distance for vehicle_1: 12124.873632656925 km
Route for vehicle_1: ['Montgomery', 'Juneau', 'Phoenix', 'Sacramento', 'Denver', 'Montgomery_2']
Total distance for vehicle_2: 4733.485543948288 km
Route for vehicle_2: ['Montgomery', 'Little Rock', 'Hartford', 'Dover', 'Tallahassee', 'Atlanta', 'Montgomery_2']


In [9]:
# Calculate the total distance traveled by all vehicles
total_distance = 0
for v in vehicles:
    route = [capitals[0]]
    for c in capitals[1:]:
        if at_city[c, v].x > 0.5:
            route.append(c)
    total_distance += sum(distance(route[i], route[i+1]) for i in range(len(route)-1))

print(f'Total distance traveled by all vehicles: {total_distance} km')

Total distance traveled by all vehicles: 16858.359176605212 km


In [10]:
import folium

# Create a map object
mapRoute = folium.Map(location=[39, -98], zoom_start=4)

# Define a function to create a line between two cities
def add_line(city1, city2, color):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    folium.PolyLine([c1, c2], color=color, weight=2.5, opacity=1).add_to(mapRoute)

# List of colors to use for the routes
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Add a line between each pair of cities visited by the same vehicle
for i, v in enumerate(vehicles):
    cities = [city for city in capitals if m.getVarByName('at_city[%s,%s]' % (city, v)).X > 0.5]
    for city1, city2 in zip(cities[:-1], cities[1:]):
        add_line(city1, city2, colors[i])

# Display the map
mapRoute
